<a href="https://colab.research.google.com/github/rickiepark/fine-tuning-llm/blob/main/Chapter1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1장 LLM 소개

### 스포일러

이 장에서는 다음과 같은 내용을 배웁니다.

- 언어 모델(language model)의 간략한 역사
- 트랜스포머(Transformer) 구조의 기본 구성 요소와 어텐션 메커니즘(attention mechanism)
- 여러 종류의 미세 튜닝 방법

### 트랜스포머

![](https://github.com/dvgodoy/FineTuningLLMs/blob/main/images/ch1/stacked_layers.png?raw=True)
<center>그림 1.1 트랜스포머 층의 스택(stack)</center>

![](https://github.com/dvgodoy/FineTuningLLMs/blob/main/images/ch1/full_transformer.png?raw=True)
<center>그림 1.2 자세한 트랜스포머 구조</center>

![](https://github.com/dvgodoy/FineTuningLLMs/blob/main/images/ch1/bert_embeddings.png?raw=True)
<center>그림 1.3 BERT의 문맥을 고려한 단어 임베딩</center>

### Attention Is All You Need

$$
\Large
\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right)V
$$
<center>식 1.1 어텐션 공식</center>

![](https://github.com/dvgodoy/FineTuningLLMs/blob/main/images/ch1/translation_att.png?raw=True)
<center>그림 1.4 어텐션 점수</center>

![](https://github.com/dvgodoy/FineTuningLLMs/blob/main/images/ch1/multiple_keys_context.png?raw=True)
<center>그림 1.5 2차원 쿼리와 키</center>

$$
\Large
\text{cos}\theta = ||Q|| ||K|| = Q \cdot K
$$
<center>식 1.2 코사인 유사도에 노름을 곱하면 두 벡터의 점곱이 됩니다.</center>

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoTokenizer
repo_id = 'microsoft/Phi-3-mini-4k-instruct'
tokenizer = AutoTokenizer.from_pretrained(repo_id)
vocab_size = len(tokenizer)

torch.manual_seed(13)
# 임베딩과 투영 층을 만듭니다.
d_model = 1024
embedding_layer = nn.Embedding(vocab_size, d_model)
linear_query = nn.Linear(d_model, d_model)
linear_key = nn.Linear(d_model, d_model)
linear_value = nn.Linear(d_model, d_model)

In [2]:
sentence = 'Just a dummy sentence'
input_ids = tokenizer(sentence, return_tensors='pt')['input_ids']
input_ids

tensor([[ 3387,   263, 20254, 10541]])

In [3]:
embeddings = embedding_layer(input_ids)
embeddings.shape

torch.Size([1, 4, 1024])

In [4]:
# 투영
proj_key = linear_key(embeddings)
proj_value = linear_value(embeddings)
proj_query = linear_query(embeddings)
# 어텐션 점수
dot_products = torch.matmul(proj_query, proj_key.transpose(-2, -1))
scores = F.softmax(dot_products / np.sqrt(d_model), dim=-1)
scores.shape

torch.Size([1, 4, 4])

In [5]:
context = torch.matmul(scores, proj_value)
context.shape

torch.Size([1, 4, 1024])